In [ ]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [ ]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re

In [ ]:
# get doi from url
def url2doi(url, source):
    url = str(url).strip()

    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options, service=Service(GeckoDriverManager().install()))
            driver.get(url)

            # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//p[text()='Consent']"))).click()

            # try:
            #     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#formInput"))).send_keys(str(doi).strip())
            # except TimeoutException:
            #     print("Waiting for clicking consent timeout")
            # try:
            #     # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/form/button"))).click()
            #     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Get PMID"]'))).click()
            # except TimeoutException:
            #     print("Waiting for clicking button timeout")
            
            error_label = 1
        except:
            print("URL to DOI transformation failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0

    try:
        my_elem = driver.find_elements(By.XPATH, "//*[contains(text(), 'doi.org') or contains(text(), 'DOI')]")
        doi = ""
        doi = str(my_elem.find_element(By.XPATH, "i").get_attribute("innerHTML"))
    except:
        doi = np.nan
    driver.quit()
    return doi
# --------------------start of test code--------------------
# url = "https://www.tandfonline.com/doi/abs/10.1080/01616412.1985.11739692"
# doi = url2doi(url)
# print(doi)
# ---------------------end of test code---------------------

In [ ]:
def preprocess_google_shcolar_step2(source_path, output_path, start, end):
    print("Starting merging search results from Google Scholar...")

    df = pd.read_csv(source_path, header=None, sep=',')
    df.columns = ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]

    for ind in range(start, end):
        # get doi from url
        if df["full_text_url"][ind] == df["full_text_url"][ind]:
            url = str(df["full_text_url"][ind]).strip()
            source = str(df["full_text_source"][ind]).strip()
            doi = url2doi(url, source)
        else:
            doi = np.nan

        # get pmid from DOI
        if doi == doi: # there's doi
            pmid = plib.doi2pmid(doi)
        else: # doi not found
            pmid = np.nan

        # get pmcid, full_text_url, full_text_source
        if pmid != pmid: # pmid is np.nan
            pmcid = np.nan
            if doi == doi: # doi is not np.nan
                full_text_url = "https://doi.org/" + str(doi).strip()
                full_text_source = "DOI"
            else:
                full_text_url = np.nan
                full_text_source = np.nan
        else: # pmid is not np.nan
            # request the webpage
            url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
            # proxies = plib.get_proxies()
            soup = plib.request_webpage(url)
            # print(soup)

            # get pmcid
            try:
                pmcid = soup.find_all("span", {"class": "identifier pmc"})[0].find_all("a", {"class": "id-link"})[0].get_text().strip()
            except:
                pmcid = np.nan
            # print(pmcid)
            
            # get full_text_url, full_text_source
            if pmcid == pmcid: # pmcid is not np.nan
                full_text_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
                full_text_source = "PMC"
            else: # pmcid is not np.nan
                try:
                    full_text_url = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["href"].strip()
                    full_text_source = soup.find_all("div", {"class": "full-text-links-list"})[0].find_all("a", {"class": "link-item dialog-focus"})[0]["data-ga-action"].strip()
                except:
                    full_text_url = np.nan
                    full_text_source = np.nan

        columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [df["Title"][ind]],
            "full_text_url": [df["full_text_url"][ind]],
            "full_text_source": [df["full_text_source"][ind]],
            "pdf_url": [df["pdf_url"][ind]],
            "pdf_source": [df["pdf_source"][ind]]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
source_path = fpath.poten_litera_gs_processed_step1
output_path = fpath.poten_litera_gs_processed_step2
# # plib.clear_file(output_path)

# df = pd.read_csv(source_path, header=None, sep=',')
# df.columns = ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
# print(df.head(3))
# print(df.shape)
# # (905, 5)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# full_text_source = set(df['full_text_source'].tolist())
# print(full_text_source)
# # {'www.frontiersin.org', 'www.elibrary.ru', 'orca.cardiff.ac.uk', 'www.jneurosci.org', 
# # 'europepmc.org', 'www.theses.fr', 'www.biorxiv.org', 'submissions.mirasmart.com', 
# # 'royalsocietypublishing.org', 'www.science.org', 'thejns.org', 
# # 'escholarship.mcgill.ca', 'www.cambridge.org', 'movementdisorders.onlinelibrary.wiley.com', 
# # 'www.ahajournals.org', 'books.google.de', 'www.mdpi.com', 'www.sciencedirect.com', 
# # 'ieeexplore.ieee.org', 'academic.oup.com', 'www.pnas.org', 'physoc.onlinelibrary.wiley.com', 
# # 'www.jstage.jst.go.jp', 'wakespace.lib.wfu.edu', 'elibrary.ru', 'www.cabdirect.org', 
# # 'www.tandfonline.com', 'www.jpn.ca', 'jpet.aspetjournals.org', 'onlinelibrary.wiley.com', 
# # 'open.bu.edu', 'tbiomed.biomedcentral.com', 'www.liebertpub.com', 'journals.lww.com', 
# # 'agro.icm.edu.pl', 'ekja.org', 'analyticalsciencejournals.onlinelibrary.wiley.com', 
# # 'n.neurology.org', 'pubs.asahq.org', 'journals.sagepub.com', 'neuro.psychiatryonline.org', 
# # 'karger.com', 'nyaspubs.onlinelibrary.wiley.com', 'pure.mpg.de', 'elifesciences.org', 
# # 'link.springer.com', 'psycnet.apa.org', 'jnnp.bmj.com', 'www.degruyter.com', 'ajp.psychiatryonline.org', 
# # 'journals.physiology.org', 'www.nature.com', 'www.jstor.org', 'var.scholarpedia.org', 'www.eneuro.org', 
# # 'journals.plos.org', 'www.cell.com', 'www.ncbi.nlm.nih.gov', 'www.taylorfrancis.com', 
# # 'bmcneurosci.biomedcentral.com', nan, 'jamanetwork.com'}
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# # ["Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
# print(df["Title"].isnull().any().any())
# print(df["full_text_url"].isnull().any().any())
# print(df["full_text_source"].isnull().any().any())
# print(df["pdf_url"].isnull().any().any())
# print(df["pdf_source"].isnull().any().any())
# # False, True, True, True, True
# # full_text_url, full_text_source, pdf_url, pdf_source contain np.nan
# # we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# preprocess_google_shcolar_step2(source_path, output_path, 0, 1000)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------